In [14]:
# Copyright 2013-2015. The Regents of the University of California.
# All rights reserved. Use of this source code is governed by 
# a BSD-style license which can be found in the LICENSE file.
#
# Authors: 
# 2013 Martin Uecker <uecker@eecs.berkeley.edu>
# 2015 Jonathan Tamir <jtamir@eecs.berkeley.edu>


import numpy as np

def readcfl(name):
    # get dims from .hdr
    h = open(name + ".hdr", "r")
    h.readline() # skip
    l = h.readline()
    h.close()
    dims = [int(i) for i in l.split( )]

    # remove singleton dimensions from the end
    n = np.prod(dims)
    dims_prod = np.cumprod(dims)
    dims = dims[:np.searchsorted(dims_prod, n)+1]

    # load data and reshape into dims
    d = open(name + ".cfl", "r")
    a = np.fromfile(d, dtype=np.complex64, count=n);
    d.close()
    return a.reshape(dims, order='F') # column-major

	
def writecfl(name, array):
    h = open(name + ".hdr", "w")
    h.write('# Dimensions\n')
    for i in (array.shape):
            h.write("%d " % i)
    h.write('\n')
    h.close()
    d = open(name + ".cfl", "w")
    array.T.astype(np.complex64).tofile(d) # tranpose for column-major order
    d.close()

In [15]:
import h5py
import sigpy as sp

h5_dataset_brain = h5py.File('/tmp/rrsg_challenge/rawdata_brain_radial_96proj_12ch.h5', 'r')

h5_dataset_rawdata_name = list(h5_dataset_brain.keys())[0]
h5_dataset_trajectory_name = list(h5_dataset_brain.keys())[1]

trajectory = h5_dataset_brain.get(h5_dataset_trajectory_name).value
rawdata = h5_dataset_brain.get(h5_dataset_rawdata_name).value

gpu_number = -1

coor = trajectory[:2,:,:].transpose((1,2,0))
coor = sp.backend.to_device(coor,gpu_number)
rawdata_brain = rawdata[0,:,:,:].transpose((2,0,1))
rawdata_brain = sp.backend.to_device(rawdata_brain,gpu_number)

In [23]:
image = readcfl("img_igrid_brain")
image_sos = sp.util.rss(image,3)
sens_maps = image[:,:,0,:]/image_sos
sens_maps_t = sens_maps.transpose((2,0,1))
sens_maps_t = sp.backend.to_device(sens_maps_t,gpu_number)

In [25]:
np.shape(sens_maps_t)

(12, 300, 300)

In [26]:
S = sp.linop.Multiply((1,300,300),sens_maps_t)
NUFFT = sp.linop.NUFFT((12,300,300),coor)
E = NUFFT*S

In [38]:
def get_dcf(coor):
    kmax = np.amax(coor)
    coord_shape = coor.shape
    nro, ntr, ndim = coord_shape
    dcf = np.zeros((nro,ntr))
    delta_k = kmax/nro
    for i in range(nro):
        for j in range(ntr):
            if (i == 0) or (j == 0):
                dcf[i,j] = 2*np.pi * 1/ntr * delta_k**2 * 1/8
            else:
                dcf[i,j] = 2*np.pi * 1/ntr * delta_k**2 * j
    return dcf

# Intensity correction
img_shape = (1,300,300)
intensity = sp.util.rss(sens_maps_t)
I = sp.linop.Multiply(img_shape,intensity)


S = sp.linop.Multiply(img_shape,sens_maps_t)
NUFFT = sp.linop.NUFFT(S.oshape,coor)
E = NUFFT*S*I
dcf = get_dcf(coor)
D = sp.linop.Multiply(E.oshape,dcf)
A = D*E


In [42]:
np.shape(intensity)

(300, 300)